In [4]:
import pandas as pd
import os
import random
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
data_dir = '/opt/ml/input/data/'
csv_file_path = os.path.join(data_dir, 'train_data_user_fe.csv')
df = pd.read_csv(csv_file_path)
df.T

,0,1,2,3,4,5,6,7,8,9,...,2266576,2266577,2266578,2266579,2266580,2266581,2266582,2266583,2266584,2266585
userID,0,0,0,0,0,0,0,0,0,0,...,7440,7441,7441,7441,7441,7441,7441,7441,7441,7441
assessmentItemID,A060001001,A060001002,A060001003,A060001004,A060001005,A060001007,A060003001,A060003002,A060003003,A060003004,...,A030197005,A030071001,A030071002,A030071003,A030071004,A030071005,A040165001,A040165002,A040165003,A040165004
testId,A060000001,A060000001,A060000001,A060000001,A060000001,A060000001,A060000003,A060000003,A060000003,A060000003,...,A030000197,A030000071,A030000071,A030000071,A030000071,A030000071,A040000165,A040000165,A040000165,A040000165
answerCode,1,1,1,1,1,1,0,1,1,1,...,0,0,0,1,0,0,1,1,1,1
Timestamp,1585009031,1585009034,1585009042,1585009049,1585009056,1585009067,1585201923,1585201930,1585201994,1585202009,...,1603269200,1591339601,1591339723,1591339763,1591339797,1591339821,1597971999,1597972010,1597972056,1597972129
KnowledgeTag,7224,7225,7225,7225,7225,7225,7226,7226,7226,7226,...,1984,438,438,438,438,438,8836,8836,8836,8836
year,2020,2020,2020,2020,2020,2020,2020,2020,2020,2020,...,2020,2020,2020,2020,2020,2020,2020,2020,2020,2020
month,3,3,3,3,3,3,3,3,3,3,...,10,6,6,6,6,6,8,8,8,8
day,24,24,24,24,24,24,26,26,26,26,...,21,5,5,5,5,5,21,21,21,21
hour,0,0,0,0,0,0,5,5,5,5,...,8,6,6,6,6,6,1,1,1,1


In [6]:
def item_fe(df):
    df['item_ans_1'] = df[df['userID'].shift(1) == df['userID']].groupby('assessmentItemID')['answerCode'].transform(lambda x:x.cumsum().shift(1))
    df['item_total_ans'] = df.groupby('assessmentItemID')['answerCode'].cumcount()
    df['item_acc'] = df['item_ans_1']/df['item_total_ans']
    
    ## item의 평균 정답률, 정답 총합, 표준편차
    correct_a = df.groupby(['assessmentItemID'])['answerCode'].agg(['mean', 'sum','std'])
    correct_a.columns = ["item_ans_mean", 'item_ans_sum','item_ans_std']
    df = pd.merge(df, correct_a, on=['assessmentItemID'], how="left")
    
    ## item의 평균 풀이 시간, 표준편차
    time_a = df.groupby(['assessmentItemID'])['time'].agg(['mean', 'std'])
    time_a.columns = ["item_time_mean", 'item_time_std']
    df = pd.merge(df, time_a, on=['assessmentItemID'], how="left")

    
    # ## item 난이도
    # df['item_lvl'] = df['item_time_mean'] / df['item_ans_mean']
    # # 범주화
    # item_cat_num = 10
    # df['item_lvl_cat'] = pd.qcut(df['item_lvl'],item_cat_num,labels=[i for i in range(item_cat_num)])

    ## item 노출 횟수
    df['item_total_answer'] = df.groupby('assessmentItemID')['answerCode'].cumcount()
    
    df = df.fillna(0)
    return df

In [7]:
df = item_fe(df)

In [9]:
df[df['userID'].shift(1) != df['userID']]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,year,month,day,hour,...,user_lvl,item_ans_1,item_total_ans,item_acc,item_ans_mean,item_ans_sum,item_ans_std,item_time_mean,item_time_std,item_total_answer
0,0,A060001001,A060000001,1,1585009031,7224,2020,3,24,0,...,40.409421,0.0,0,0.0,0.982063,219,0.133022,13.031390,28.253899,0
745,1,A040013001,A040000013,1,1578300043,2048,2020,1,6,8,...,202.604036,0.0,0,0.0,0.800000,212,0.400757,58.007547,64.048513,0
1678,2,A030050001,A030000050,1,1578654173,407,2020,1,10,11,...,121.351175,0.0,0,0.0,0.829545,219,0.376746,55.647727,71.766633,0
1954,5,A080001001,A080000001,1,1581537620,4604,2020,2,12,20,...,102.687234,0.0,0,0.0,0.480447,172,0.500317,27.846369,49.705140,0
2787,6,A030016001,A030000016,1,1583750908,7307,2020,3,9,10,...,136.395749,0.0,0,0.0,0.921642,247,0.269237,44.402985,57.203509,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266515,7436,A050095001,A050000095,1,1583217245,360,2020,3,3,6,...,89.177357,0.0,227,0.0,0.885965,202,0.318553,44.179825,53.265590,227
2266530,7437,A040072001,A040000072,1,1585035915,2076,2020,3,24,7,...,90.055739,0.0,265,0.0,0.932331,248,0.251651,37.488722,59.238006,265
2266546,7438,A080002001,A080000002,1,1582664394,4605,2020,2,25,20,...,166.713663,0.0,221,0.0,0.828829,184,0.377510,63.612613,72.888953,221
2266562,7440,A050096001,A050000096,1,1597813039,5267,2020,8,19,4,...,122.098803,0.0,224,0.0,0.928889,209,0.257583,81.648889,80.972638,224


In [18]:
df['item_acc']

0          1.000000
1          1.000000
2          1.000000
3          1.000000
4          1.000000
             ...   
2266581    0.435252
2266582    0.649635
2266583    0.635036
2266584    0.791971
2266585    0.543796
Name: item_acc, Length: 2266586, dtype: float64